## Teoria de Números Computacional
### Trabalho 2 - Diffie-Hellman key exchange

#### Hugo Sousa (a76257 - LCC)
#### Matias Capitão (a82726 - LCC) 
#### Rafael Antunes (a77457 - LCC)

### Motivação 

Se a Alice e o Bob querem comunicar por um canal inseguro e concordam em usar uma cifra simétrica, põe-se então a seguinte questão:
    **Como é que a Alice e o Bob podem partilhar a chave secreta?**.

Este é conhecido como o _"problema de distribuição de chaves"_.

O método de troca de chaves _**Diffie-Hellman**_ resolve este problema. Este método é uma aplicação do problema do logaritmo discreto no grupo multiplicativo _**\<g> = $Z^*_p$**_ onde g é uma raiz primitiva de p.

### Protocolo de configuração do Diffie-Hellman

**1.** Escolher um número primo grande **p**.

**2.** Escolher um **g**, tal que **g** é uma raiz primitiva módulo **p**, mais concretamente **g** é um gerador de $Z^*_p$ (grupo múltiplicativo módulo p)

**3.** Publicar **p** e **g**

### Protocolo de troca de chaves do Diffie-Hellman

Como p e g são públicos Alice e Bob vão efetuar os seguintes passos:

**1.** Alice e Bob escolhem um número aleatório respectivamente **a** e **b** de **{1,...,p-1}**, este número irá ser privado.

**2.** Alice vai computar $A = g^a mod p$ e Bob vai computar $B = g^b mod p$

**3.** Alice manda  **A** para Bob pelo canal inseguro, e Bob manda **B** para Alice pelo canal inseguro

**4.** Alice usa **B** que recebeu do Bob e computa $B^a mod p$ e Bob usa **A** que recebeu de Alice e computa $A^b mod p$

**5.** O resultado dessas duas operações é igual, pois $ =A^b mod p = (g^a mod p)^b = g^ab mod p = (g^b mod p)^a = B^a mod p = $ **k**.  

Então **k** será a chave usada para a cifra simétrica da comunicação entre Alice e Bob.







![alt text][diagram]

[diagram]: https://www.researchgate.net/profile/Mandrita_Mondal/publication/332368903/figure/fig2/AS:746707394498561@1555040324478/Pictorial-representation-of-Diffie-Hellman-key-exchange-protocol.ppm "DHKE"

### Codificação

In [22]:
import random

class DHKE:
    def __init__(self, n):
        aux = random_prime(2^n,2^n+1)
        self.p = aux
        self.g = primitive_root(aux)

    def publish(self):
        print(self.p, self.g)

    def gen_secret(self):
        return(random.randint(1,(self.p)-1))

    def compute(self, m, n):
        return(power_mod(m, n, self.p))

##### Exemplo 1: Alice e Bob querem comunicar

In [23]:
# Alice e Bob "escolhem" um primo p(um grupo multiplicativo módulo p) e um gerador g(raiz primitiva módulo p)

# n é a seed do gerador dos dois números
n = 128

#Os dois inteiros são gerados
D = DHKE(n)

#Os dois inteiros são_
D.publish()

(304164752673956602752490430774504350901, 2)


In [24]:
#A Alice escolhe um número secreto entre {1,..,p-1}

a = D.gen_secret()
a

114189641299888238748071067107244543443L

In [25]:
#O Bob escolhe um número secreto entre {1,..,p-1}
b = D.gen_secret()
b

284300746941436845758254014745327062462L

In [26]:
#Alice computa o numero que gerou e envia a Bob
A = D.compute(D.g, a)

#Bob computa o numero que gerou e envia a Alice
B = D.compute(D.g, b)

In [27]:
#Alice computa o numero que recebeu de Bon
sA = D.compute(B, a)

#Bob computa o numero que recebeu de Alice
sB = D.compute(A, b)

if sA == sB:
    print("A chave secreta 'k' é -> " + str(sA) + " pode ser usada para a encriptação simétrica")

A chave secreta 'k' é -> 32954024766508317600969666952462235254 pode ser usada para a encriptação simétrica


#### Man in the Middle Attack

Como explicamos tanto os valores de **p** e de **g**, como os valores de $A = g^a mod p$ e $B = g^b mod p$, são públicos, são por isso sujeitos a atacantes.

Seja o atacante chamado Chuck, a questão que aqui se coloca é: _**será que Chuck consegue descobrir o valor da chave secreta 'k' partilhada por Alice e Bob, a partir dos valores que são públicos?**_

Ora, em exemplos muito simples é possível descobrir-se o valor de **a** e **b** e consequentemente de **k**, mas em prática quando **p** é muito grande e os expoentes da exponenciação modular também o são, embora os computadores conseguem resolver os cálculos bastante rápido, o processo inverso é muito difícil de computar, é neste ponto que o protocolo de troca de chaves Diffie-Hellman toma proveito do problema do logaritmo discreto.

Pois, seja N um número primo muito grande o grupo multiplicativo $Z^*_N$ tem exatamente N-1 elementos, o que faria com que **Chuck** precisa-se de percorrer enúmeros geradores e potências desses geradores de $Z^*_N$ para encontrar resposta, quando N tem centenas de dígitos ou mais, o problema do logaritmo discreto torna-se computacionalmente inviável.

No entanto, mesmo que Chuck não consiga descobrir os valores secretos de Alice e Bob existem formas de Chuck tomar partido deste protocolo, nomeadamente se Chuck interceptar os valores **A** e **B** e sabendo os valores públicos de **p** e de **g**, Chuck pode mandar os seus próprios valores públicos tanto a Alice como a Bob e assim pode furjar duas chaves: **k1** que irá partilhar com Alice, e **k2** que irá partilhar com Bob, a partir daqui Chuck pode ver as mensagens enviadas e possivelmente modificá-las.

![alt text][MITM]

[MITM]: https://i.imgur.com/Cq78TET.png?1 "MITM"

#### Exemplo 2: Chuck: The man in the middle

In [28]:
# Alice e Bob "escolhem" um primo p(um grupo multiplicativo módulo p) e um gerador g(raiz primitiva módulo p)

# n é a seed do gerador dos dois números
n = 128

#Os dois inteiros são gerados
D = DHKE(n)

#Os dois inteiros são_
D.publish()

#A Alice escolhe um número secreto entre {1,..,p-1}
a = D.gen_secret()

#O Bob escolhe um número secreto entre {1,..,p-1}
b = D.gen_secret()



(222153088566641259361561068696378978821, 2)


In [29]:
#Alice computa o numero que gerou e envia a Bob
A = D.compute(D.g, a)

#Bob computa o numero que gerou e envia a Alice
B = D.compute(D.g, b)

In [30]:
#Neste ponto Chuck que está a ver a comunicação e está ciente dos valores públicos vai computar os seus valores públicos e privados

#O Chuck escolhe um número secreto entre {1,..,p-1}
c = D.gen_secret()

#Chuck computa o numero que gerou e e troca-o pelo valor que Alice estava a tentar enviar a Bob, e envia essa valor a Bob
cA = D.compute(D.g, c)

#Chuck computa o numero que gerou e e troca-o pelo valor que Bob estava a tentar enviar a Alice, e envia esse valor a Alice
cB = D.compute(D.g, c)

In [31]:
#Alice computa o numero que recebeu de Chuck pensando ser de Bob
k1 = D.compute(cB, a)

#Chuck computa o numero que recebeu de Alice
k11 =D.compute(A, c)

if(k1==k11):
    print("Chuck tem agora acesso às mensagens de Alice!!")

#Bob computa o numero que recebeu de Chuck pensando ser de Alice
k2 = D.compute(cA, b)

#Chuck computa o numero que recebeu de Bob
k22 =D.compute(B, c)

if(k2==k22):
    print("Chuck tem agora acesso às mensagens de Bob!!")

Chuck tem agora acesso às mensagens de Alice!!
Chuck tem agora acesso às mensagens de Bob!!


### Solução

O problema que aqui surge é que acordo de uma chave entre Alice e Bob não é autenticado, surge então a necessidade de de criar uma versão de Diffie-Hellman autênticada esta versão é conhecida como _**"Station-to-Station protocol"**_. Esta versão usa a noção de assinatura digital estudada no trabalho anterior, assim, embora Chuck consiga interceptar as mensagens de Alice e Bob, não consegue furjar a assinatura de Alice ou de Bob sem conhecer os valores privado de Alice ou Bob.

### Referências:

<https://www.youtube.com/watch?v=ESPT_36pUFc> : "The Mathematics of Diffie-Hellman Key Exchange | Infinite Series"

<https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange> "Diffie–Hellman key exchange Wikipedia"

<https://www.youtube.com/watch?v=pwZebAcjoeo&list=PL1xkDS1G9As4Yz_te20j1A9evIjt5Z06e&index=118> : "Applied Cryptography: Diffie–Hellman Key Exchange - Part 1"

William Stein : "Elementary Number Theory:Primes, Congruences, and Secrets"

<https://stackoverflow.com/questions/10471009/how-does-the-man-in-the-middle-attack-work-in-diffie-hellman>